In [0]:
%sql
DROP TABLE gizmobox.bronze.customers_autoloader;

In [0]:
customers_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", "/Volumes/gizmobox/landing/operational_data/customers_autoloader/_schema")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaHints", "customer_id LONG, created_timestamp TIMESTAMP, date_of_birth DATE, member_since DATE, email STRING, telephone STRING")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
    .load("/Volumes/gizmobox/landing/operational_data/customers_autoloader/")
)

In [0]:
import pyspark.sql.functions as F

customers_transformed_df=customers_df.withColumn("file_path", F.col("_metadata.file_path")).withColumn(
    "ingestion_date", F.current_timestamp()
)

In [0]:
streaming_query = (
    customers_transformed_df.writeStream.format("delta")
    .option(
        "checkpointLocation",
        "/Volumes/gizmobox/landing/operational_data/customers_autoloader/_checkpoint_stream",
    )
    .option("mergeSchema", "true")
    .toTable("gizmobox.bronze.customers_autoloader")
)

In [0]:
%sql
select * from gizmobox.bronze.customers_autoloader;

In [0]:
streaming_query.stop()